In [1]:
import re
import copy 
import json
import math
import itertools
import jellyfish
from tqdm import tqdm
import pandas as pd
import numpy as np
from datetime import datetime
import xml.etree.ElementTree as ET

ns = {'xml': 'http://www.w3.org/XML/1998/namespace',
      'dflt': 'http://www.tei-c.org/ns/1.0',
      'frus':'http://history.state.gov/frus/ns/1.0',
      'xi':'http://www.w3.org/2001/XInclude'
      }

In [2]:
def extract_city(doc):

    # city
    place_tag = doc.find('.//dflt:placeName',ns)
    if place_tag is not None:
        txt = "".join(place_tag.itertext())
        city = " ".join(txt.split())
    else:
        city = None

    global city_df
    city_df = pd.concat((city_df, pd.DataFrame({'name':[city]})),ignore_index=True)
    return

In [3]:
import glob
volume_root = 'frus1969-76'

city_df = pd.DataFrame(columns=['name'])


#for file in glob.glob('volumes/*'):
for file in glob.glob('volumes/'+volume_root+'*'):

    tree = ET.parse(file)
    root = tree.getroot()

    docs = root.findall('./dflt:text/dflt:body//dflt:div[@type="document"]', ns)
    for doc in docs:
        extract_city(doc)


In [ ]:
#pd.set_option('display.max_rows', None)

In [4]:
city_df.dropna(inplace=True)
city_df.drop_duplicates(inplace=True)
city_df.reset_index(drop=True,inplace=True)

In [5]:
extension_col = city_df['name'].apply(lambda x: " ".join(x.split(',')[1:]))
name_col = city_df['name'].apply(lambda x: x.split(',')[0])
city_df['name'] = name_col
city_df['extension'] = extension_col
city_df['extension'] = city_df['extension'].apply(lambda x: None if len(x)==0 else x)

## wc matching

In [6]:
wc_df = pd.read_csv('world-cities.csv')

In [7]:
# helpers
def geo_match(pattern,string):
    
    if pattern !=pattern:
        return None
    elif re.search(pattern,string):
        return pattern
    else:
        return None

def f(string):

    if not string:
        return None
    
    tl = list(wc_df[wc_df['country'].apply(lambda pattern: True if geo_match(pattern,string) else False)].drop_duplicates(subset='country')['country'].values)
    if len(tl)==0:
        tl = list(wc_df[wc_df['subcountry'].apply(lambda pattern: True if geo_match(pattern,string) else False)].drop_duplicates(subset='country')['country'].values)

    if len(tl)==0:
        return None
    elif len(tl)==1:
        return tl[0]
    else:
        print(f'multi-match for {string}. Check later!')
        return tl

def f2(string):

    if not string:
        return None

    tl = list(wc_df[wc_df['subcountry'].apply(lambda pattern: True if geo_match(pattern,string) else False)].drop_duplicates(subset='country')['country'].values)

    if len(tl)==0:
        return None
    elif len(tl)==1:
        return tl[0]
    else:
        print(f'multi-match for {string}. Check later!')
        return tl

def merger(row):

    d1 = row['extension_match']
    d2 = row['wc_guess']

    if not d1 and d2!=d2:
        return None
    elif not d1:
        return d2
    else:
        return d1

In [8]:
city_df['extension_match'] = city_df['extension'].apply(lambda x:f(x))
city_df['wc_guess'] = city_df[city_df['extension'].isna()]['name'].apply(lambda x:f2(x))
#city_df['merged']=city_df.apply(lambda x: merger(x),axis=1)
city_df['merged']=city_df['extension_match']

city_df=city_df[['name','merged','extension_match','wc_guess']]
city_df.rename(columns={'merged':'country'},inplace=True)

multi-match for  Maryland. Check later!
multi-match for  Florida. Check later!
multi-match for  Maryland. Check later!
multi-match for Salzburg White House. Check later!
multi-match for Dar es Salaam. Check later!
multi-match for Salzburg. Check later!
multi-match for La Paz. Check later!
multi-match for San José. Check later!
multi-match for San Salvador. Check later!
multi-match for Montevideo. Check later!


resolve multi-match cases by hand before proceeding next part!

In [9]:
# save and edit
city_df.to_csv('tables/city_69_76.csv')

In [10]:
# load corrected one
city_df = pd.read_csv('tables/city_69_76.csv')

### wikidata matching

In [11]:
import pandas as pd
import dask.dataframe as dd
from collections import Counter
from dask.diagnostics import ProgressBar
from SPARQLWrapper import SPARQLWrapper, JSON

from tqdm import tqdm
tqdm.pandas()

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [12]:
user_agent = 'CoolBot/0.0 (https://example.org/coolbot/; coolbot@example.org)'

sparqlwd = SPARQLWrapper("https://query.wikidata.org/sparql", agent=user_agent)
sparqlwd.setReturnFormat(JSON)

In [13]:
def find_if_capital(name):

    try:
        query = """
        SELECT ?country ?countryLabel WHERE {
        SERVICE wikibase:mwapi {
            bd:serviceParam wikibase:endpoint "www.wikidata.org";
                            wikibase:api "EntitySearch";
                            mwapi:search  \'"""+name+"""\';
                            mwapi:language "en".
            ?city wikibase:apiOutputItem mwapi:item.
            ?num wikibase:apiOrdinal true.
        }
        ?city wdt:P31 wd:Q5119.
        ?city wdt:P17 ?country.
        SERVICE wikibase:label { bd:serviceParam wikibase:language "en".}
        }
        """
        
        sparqlwd.setQuery(query)

        return sparqlwd.query().convert()

    except Exception as e:
        print(f'name: {name}')
        print(f'error message: {e}')
        return {'head': {'vars': ['item']}, 'results': {'bindings': []}}


def find_if_bigcity(name):

    try:
        query = """
        SELECT ?country ?countryLabel WHERE {
        SERVICE wikibase:mwapi {
            bd:serviceParam wikibase:endpoint "www.wikidata.org";
                            wikibase:api "EntitySearch";
                            mwapi:search  \'"""+name+"""\';
                            mwapi:language "en".
            ?city wikibase:apiOutputItem mwapi:item.
            ?num wikibase:apiOrdinal true.
        }
        ?city (wdt:P31/wdt:P279*) wd:Q1549591.
        ?city wdt:P17 ?country.
        SERVICE wikibase:label { bd:serviceParam wikibase:language "en".}
        }
        """
        
        sparqlwd.setQuery(query)

        return sparqlwd.query().convert()

    except Exception as e:
        print(f'name: {name}')
        print(f'error message: {e}')
        return {'head': {'vars': ['item']}, 'results': {'bindings': []}}


def process_name(row, f):

    name = row['name']

    res = f(name)

    candidates = []
    selected_country = None
    selected_tag = None

    for binding in res['results']['bindings']:
        candidates.append(binding['countryLabel']['value'])

    if len(candidates)>0:
        temp_country = Counter(candidates).most_common(1)[0][0]
        temp_tag = None

        for binding in res['results']['bindings']:
            if binding['countryLabel']['value'] == temp_country:
                temp_tag = binding['country']['value']
                break
        
        #selected_country.add(temp_country)
        selected_country = temp_country
        #wiki_tag.add(temp_tag)
        selected_tag = temp_tag

    return selected_country

In [14]:
# find country if city is capital
wiki_df = city_df[city_df['extension_match'].apply(lambda x: True if x!=x else False)]
city_df['wiki_capital_guess'] = wiki_df.apply(process_name,axis=1,f=find_if_capital)

# find country if city is big city but not capital
wiki_df = city_df[city_df['extension_match'].apply(lambda x: True if x!=x else False) & city_df['wiki_capital_guess'].apply(lambda x: False if x else True)]
city_df['wiki_bigcity_guess'] = wiki_df.apply(process_name,axis=1,f=find_if_bigcity)

In [15]:
def merger2(row):

    d1 = row['wiki_capital_guess']
    d2 = row['wiki_bigcity_guess']

    if (not d2 or d2!=d2) and (not d1 or d1!=d1):
        return None
    elif (not d2 or d2!=d2):
        return d1
    else:
        return d2

city_df['merged_wiki'] = city_df.apply(merger2,axis=1)

In [16]:
def merger3(row):

    d1 = row['country']
    d2 = row['merged_wiki']

    if not d2 and d1!=d1:
        return None
    elif d1!=d1:
        return d2
    else:
        return d1

city_df['country'] = city_df.apply(merger3,axis=1)

In [18]:
def merger4(row):

    d1 = row['country']
    d2 = row['wc_guess']

    if (not d2 or d2!=d2) and (not d1 or d1!=d1):
        return None
    elif (not d1 or d1!=d1):
        return d2
    else:
        return d1

city_df['country'] = city_df.apply(merger4,axis=1)

resolve multi-match cases by hand before proceeding next part!

In [20]:
# save and edit
city_df.to_csv('tables/city_69_76.csv')

In [175]:
# load corrected one
city_df = pd.read_csv('tables/city_69_76.csv')

## misspelling matching

In [176]:
city_df=city_df[['name','country']]

In [177]:
all_names = city_df['name'].values

def compute_sim(s1,func,s2):
    return func(s1,s2)

def find_matches(s2):

    spiro_dist_df = pd.DataFrame({'name_set':all_names,
                                'dam_lev_dist':[compute_sim(x, jellyfish.damerau_levenshtein_distance,s2) for x in all_names]})
    
    misspelling_idx = set(spiro_dist_df[(spiro_dist_df['dam_lev_dist'] <=1)].index.values)

    return misspelling_idx

In [178]:
t = {}
for idx in tqdm(range(len(all_names))):
    name = all_names[idx]
    t[idx]=find_matches(name)

100%|██████████| 238/238 [00:00<00:00, 1020.49it/s]


In [179]:
scratch_t = copy.deepcopy(t)
changed_flag = True

while changed_flag:

    changed_flag = False

    for key in t:
        
        for matched_idx in t[key]:

            if key != matched_idx:
                if scratch_t.get(key, None) and scratch_t.get(matched_idx, None):
                    changed_flag = True
                    t[key] = t[key].union(t[matched_idx])
                    scratch_t.pop(matched_idx, None)
        
    unwanted = set(t.keys()) - set(scratch_t.keys())
    print(f'removing {len(unwanted)} keys.')
    for unwanted_key in unwanted: del t[unwanted_key]
    scratch_t = copy.deepcopy(t)
    print('---')
    

removing 25 keys.
---
removing 0 keys.
---


In [180]:
for temp_key in t:
    
    te_df = city_df.iloc[list(t[temp_key])]

    name_list = te_df['name'].values

    country_list = te_df['country'].values
    country_list = [c for c in country_list if c==c]
    country_list = list(set(country_list))
    if len(country_list)==0:
        country_list = None
    elif len(country_list)==1:
        country_list = country_list[0]

    city_df.at[temp_key, 'name_list'] = name_list
    city_df.at[temp_key, 'country'] = country_list

city_df = city_df.loc[t.keys()]

In [149]:
city_df.to_csv('tables/city_69_76_misspelling.csv')

city_df[['name_list']].to_parquet('tables/city_69_76_misspelling_namelist.parquet')

In [44]:
# now edit and correct the file into its final format
pd.set_option('display.max_rows', None)
city_df

,name,country,name_list
0,Washington,United States of America,"[Washington, Wasington, Washington, Washingon]"
1,Moscow,Russia,[Moscow]
2,Vienna,Austria,[Vienna]
3,Camp David,United States,"[Camp David, Camp David]"
4,Salzburg White House,"['Austria', 'Cape Verde']",[Salzburg White House]
5,Salzburg,Austria,"[Salzburg, Salzburg]"
6,Helsinki,Finland,"[Helsinki, Helsinki]"
7,Cape Town,South Africa,[Cape Town]
8,Pretoria,South Africa,[Pretoria]
9,New York,United States of America,"[New York, New York]"


In [193]:
pd.reset_option('display.max_rows')

city_df = pd.read_csv('tables/city_69_76_misspelling.csv')

name_list_df = pd.read_parquet('tables/city_69_76_misspelling_namelist.parquet').reset_index(drop=True)

city_df['name_list']=name_list_df['name_list']

In [194]:
def name_converter(name):

    if name == 'United States':
        return 'United States of America'
    elif name == 'China':
        return "People's Republic of China"
    elif name == 'Martinique':
        return 'France'
    elif name == 'Bermuda':
        return 'United Kingdom'
    elif name == 'Puerto Rico':
        return 'United States of America'
    elif name == 'Republic of Vietnam':
        return 'Vietnam'
    else:
        return name

city_df['country'] = city_df['country'].apply(name_converter)

In [195]:
# only for matching city names with doc_df
city_lookup_dict = {} # 'misspelled':'corrected'

In [196]:
for _, row in city_df.iterrows():

    for misspelled_name in row['name_list']:
        
        if misspelled_name not in city_lookup_dict:
            city_lookup_dict[misspelled_name] = row['name']

In [197]:
city_lookup_dict

{'Washington': 'Washington',
 'Wasington': 'Washington',
 'Washingon': 'Washington',
 'Moscow': 'Moscow',
 'Vienna': 'Vienna',
 'Camp David': 'Camp David',
 'Salzburg White House': 'Salzburg White House',
 'Salzburg': 'Salzburg',
 'Helsinki': 'Helsinki',
 'Cape Town': 'Cape Town',
 'Pretoria': 'Pretoria',
 'New York': 'New York',
 'London': 'London',
 'Yaounde': 'Yaounde',
 'Oeiras': 'Oeiras',
 'Kinshasa': 'Kinshasa',
 'Lourenco Marques': 'Lourenco Marques',
 'Lisbon': 'Lisbon',
 'Luanda': 'Luanda',
 'Paris': 'Paris',
 'Tehran': 'Tehran',
 'Jidda': 'Jidda',
 'Brussels': 'Brussels',
 'Boston': 'Boston',
 'Stockholm': 'Stockholm',
 'Tanzania': 'Tanzania',
 'Dar es Salaam': 'Dar es Salaam',
 'Lusaka': 'Lusaka',
 'Grafenau': 'Grafenau',
 'Zurich': 'Zurich',
 'Geneva': 'Geneva',
 'Bonn': 'Bonn',
 'san Clemente': 'San Clemente',
 'San Clemente': 'San Clemente',
 'Warsaw': 'Warsaw',
 'Sofia': 'Sofia',
 'Prague': 'Prague',
 'Budapest': 'Budapest',
 'Bucharest': 'Bucharest',
 'Belgrade': 'Belgr

In [199]:
city_df = city_df[['name','country','name_list']]
#city_df.to_csv('tables/city_69_76_final.csv')
city_df.to_parquet('tables/city_69_76_final.parquet')

json = json.dumps(city_lookup_dict)
f = open("tables/city_lookup_dict.json","w")
f.write(json)
f.close()

### create country-wikiTag dataframe

In [96]:
def find_country(name):

    try:
        query = """
        SELECT ?country ?countryLabel WHERE {
        SERVICE wikibase:mwapi {
            bd:serviceParam wikibase:endpoint "www.wikidata.org";
                            wikibase:api "EntitySearch";
                            mwapi:search  \""""+name+"""\";
                            mwapi:language "en".
            ?country wikibase:apiOutputItem mwapi:item.
            ?num wikibase:apiOrdinal true.
        }
        ?country wdt:P31 wd:Q6256.
        SERVICE wikibase:label { bd:serviceParam wikibase:language "en".}
        }
        """
        
        sparqlwd.setQuery(query)

        return sparqlwd.query().convert()

    except Exception as e:
        print(f'name: {name}')
        print(f'error message: {e}')
        return {'head': {'vars': ['item']}, 'results': {'bindings': []}}

In [118]:
def process_name2(name):

    res = find_country(name)

    if len(res['results']['bindings'])>0:
        binding = res['results']['bindings'][0]

        country = binding['countryLabel']['value']
        tag = binding['country']['value']

        return country, tag
    
    else:
        return name, None

In [119]:
country_tag_pairs = list(map(process_name2,city_df['country'].unique()))

countryLabel = list(map(lambda x:x[0],country_tag_pairs))
countryTag = list(map(lambda x:x[1],country_tag_pairs))

name: nan
error message: can only concatenate str (not "float") to str


In [120]:
country_df = pd.DataFrame.from_dict({'countryLabel':countryLabel,'countryTag':countryTag})


In [122]:
country_df.to_csv('tables/country_69_76.csv')